<a href="https://colab.research.google.com/github/AnkitaSK/lab-text-generation-shakespeare/blob/main/NLP_Week4_Exercise_Shakespeare_Answer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab | Text Generation from Shakespeare's Sonnet

This notebook explores the fascinating domain of text generation using a deep learning model trained on Shakespeare's sonnets.

The objective is to create a neural network capable of generating text sequences that mimic the style and language of Shakespeare.

By utilizing a Recurrent Neural Network (RNN) with Long Short-Term Memory (LSTM) layers, this project aims to demonstrate how a model can learn and replicate the complex patterns of early modern English.

The dataset used consists of Shakespeare's sonnets, which are preprocessed and tokenized to serve as input for the model.

Throughout this notebook, you will see the steps taken to prepare the data, build and train the model, and evaluate its performance in generating text.

This lab provides a hands-on approach to understanding the intricacies of natural language processing (NLP) and the potential of machine learning in creative text generation.

Let's import necessary libraries

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku
import numpy as np

Let's get the data!

In [ ]:
import requests
url = 'https://raw.githubusercontent.com/martin-gorner/tensorflow-rnn-shakespeare/master/shakespeare/sonnets.txt'
resp = requests.get(url)
with open('sonnets.txt', 'wb') as f:
    f.write(resp.content)

data = open('sonnets.txt').read()

corpus = data.lower().split("\n")

In [ ]:
data

"\tSONNETS\n\n\n\nTO THE ONLY BEGETTER OF\nTHESE INSUING SONNETS\nMR. W. H. ALL HAPPINESS\nAND THAT ETERNITY\nPROMISED BY\nOUR EVER-LIVING POET WISHETH\nTHE WELL-WISHING\nADVENTURER IN\nSETTING FORTH\nT. T.\n\n\nI.\n\nFROM fairest creatures we desire increase,\nThat thereby beauty's rose might never die,\nBut as the riper should by time decease,\nHis tender heir might bear his memory:\nBut thou, contracted to thine own bright eyes,\nFeed'st thy light'st flame with self-substantial fuel,\nMaking a famine where abundance lies,\nThyself thy foe, to thy sweet self too cruel.\nThou that art now the world's fresh ornament\nAnd only herald to the gaudy spring,\nWithin thine own bud buriest thy content\nAnd, tender churl, makest waste in niggarding.\n  Pity the world, or else this glutton be,\n  To eat the world's due, by the grave and thee.\n\nII.\n\nWhen forty winters shall beseige thy brow,\nAnd dig deep trenches in thy beauty's field,\nThy youth's proud livery, so gazed on now,\nWill be a 

In [ ]:
corpus

['\tsonnets',
 '',
 '',
 '',
 'to the only begetter of',
 'these insuing sonnets',
 'mr. w. h. all happiness',
 'and that eternity',
 'promised by',
 'our ever-living poet wisheth',
 'the well-wishing',
 'adventurer in',
 'setting forth',
 't. t.',
 '',
 '',
 'i.',
 '',
 'from fairest creatures we desire increase,',
 "that thereby beauty's rose might never die,",
 'but as the riper should by time decease,',
 'his tender heir might bear his memory:',
 'but thou, contracted to thine own bright eyes,',
 "feed'st thy light'st flame with self-substantial fuel,",
 'making a famine where abundance lies,',
 'thyself thy foe, to thy sweet self too cruel.',
 "thou that art now the world's fresh ornament",
 'and only herald to the gaudy spring,',
 'within thine own bud buriest thy content',
 'and, tender churl, makest waste in niggarding.',
 '  pity the world, or else this glutton be,',
 "  to eat the world's due, by the grave and thee.",
 '',
 'ii.',
 '',
 'when forty winters shall beseige thy b

Step 1: Initialise a tokenizer and fit it on the corpus variable using .fit_on_texts

In [ ]:
# Your code here :

tokenizer = Tokenizer(char_level=False)
tokenizer.fit_on_texts(corpus)

print("Character-to-Index Mapping:", tokenizer.word_index)

Character-to-Index Mapping: {'and': 1, 'the': 2, 'to': 3, 'of': 4, 'my': 5, 'i': 6, 'in': 7, 'that': 8, 'thy': 9, 'thou': 10, 'with': 11, 'for': 12, 'is': 13, 'not': 14, 'love': 15, 'but': 16, 'a': 17, 'me': 18, 'thee': 19, 'so': 20, 'be': 21, 'as': 22, 'all': 23, 'you': 24, 'his': 25, 'which': 26, 'when': 27, 'it': 28, 'this': 29, 'by': 30, 'your': 31, 'doth': 32, 'do': 33, 'from': 34, 'on': 35, 'or': 36, 'no': 37, 'then': 38, 'have': 39, 'what': 40, 'are': 41, 'if': 42, 'more': 43, 'mine': 44, 'their': 45, 'shall': 46, 'sweet': 47, 'time': 48, 'will': 49, 'they': 50, 'beauty': 51, 'nor': 52, 'eyes': 53, 'art': 54, 'her': 55, 'heart': 56, 'yet': 57, 'o': 58, 'than': 59, 'can': 60, 'should': 61, 'thine': 62, 'now': 63, 'where': 64, 'make': 65, 'one': 66, 'hath': 67, 'he': 68, 'fair': 69, 'still': 70, 'how': 71, 'eye': 72, 'him': 73, 'like': 74, 'true': 75, 'see': 76, 'am': 77, 'she': 78, 'those': 79, 'though': 80, 'being': 81, 'some': 82, 'every': 83, 'such': 84, 'own': 85, 'were': 86,

Step 2: Calculate the Vocabulary Size

Let's figure out how many unique words are in your corpus. This will be the size of your vocabulary.

Calculate the length of tokenizer.word_index, add 1 to it and store it in a variable called total_words.

In [ ]:
# Your code here :
total_words = len(tokenizer.word_index) + 1 # Adding 1 because indexing starts at 1
print("Vocabulary Size:", total_words)

Vocabulary Size: 3375


Create an empty list called input_sequences.

For each sentence in your corpus, convert the text into a sequence of integers using the tokenizer.
Then, generate n-gram sequences from these tokens.

Store the result in the list input_sequences.

In [ ]:
import nltk

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Your code here :
from nltk.tokenize import word_tokenize

if isinstance(corpus, list):
    corpus = ' '.join(corpus)

sentences = nltk.sent_tokenize(corpus)
print(sentences)

# convert the text into a sequence of integers using the tokenizer
sentence_sequences = tokenizer.texts_to_sequences(sentences)
print("Integer sequences for sentences:", sentence_sequences[:5])

# Then, generate n-gram sequences from these tokens.
input_sequences = []

for token_list in sentence_sequences:
  for i in range(1,len(token_list) + 1):
    n_gram_sequence = token_list[:i]
    input_sequences.append(n_gram_sequence)

print("Sample n-gram sequences:", input_sequences[:10])  # Display first 10 sequences
print("Total n-gram sequences generated:", len(input_sequences))

["\tsonnets    to the only begetter of these insuing sonnets mr. w. h. all happiness and that eternity promised by our ever-living poet wisheth the well-wishing adventurer in setting forth t. t.   i.  from fairest creatures we desire increase, that thereby beauty's rose might never die, but as the riper should by time decease, his tender heir might bear his memory: but thou, contracted to thine own bright eyes, feed'st thy light'st flame with self-substantial fuel, making a famine where abundance lies, thyself thy foe, to thy sweet self too cruel.", "thou that art now the world's fresh ornament and only herald to the gaudy spring, within thine own bud buriest thy content and, tender churl, makest waste in niggarding.", "pity the world, or else this glutton be,   to eat the world's due, by the grave and thee.", 'ii.', "when forty winters shall beseige thy brow, and dig deep trenches in thy beauty's field, thy youth's proud livery, so gazed on now, will be a tatter'd weed, of small worth

Calculate the length of the longest sequence in input_sequences. Assign the result to a variable called max_sequence_len.

Now pad the sequences using pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre').
Convert it to a numpy array and assign the result back to our variable called input_sequences.

In [ ]:
# Your code here :
# Calculate the length of the longest sequence in input_sequences. Assign the result to a variable called max_sequence_len.
max_sequence_len = max(len(seq) for seq in input_sequences)
print("Maximum sequence length:", max_sequence_len)

Maximum sequence length: 122


In [ ]:
# Pad sequences to ensure they all have the same length
input_sequences =  pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Convert to a NumPy array
input_sequences = np.array(input_sequences)

print("Padded sequences shape:", input_sequences.shape)

Padded sequences shape: (17805, 122)


Prepare Predictors and Labels

Split the sequences into two parts:

- Predictors: All elements from input_sequences except the last one.
- Labels: The last element of each sequence in input_sequences.

In [ ]:
# Your code here :
# Predictors: All elements except the last one in each sequence
predictors = input_sequences[:,:-1]

# Labels: The last element in each sequence
labels = input_sequences[:,-1]

print("Predictors shape:", predictors.shape)
print("Labels shape:", labels.shape)

Predictors shape: (17805, 121)
Labels shape: (17805,)


One-Hot Encode the Labels :

Convert the labels (which are integers) into one-hot encoded vectors.

Ensure the length of these vectors matches the total number of unique words in your vocabulary.

Use ku.to_categorical() on labels with num_classes = total_words

Assign the result back to our variable labels.

In [ ]:
# Your code here :
labels = ku.to_categorical(labels, num_classes=total_words)
print("Labels shape after one-hot encoding:", labels.shape)

Labels shape after one-hot encoding: (17805, 3375)


# Initialize the Model

Start by creating a Sequential model.

Add Layers to the Model:

Embedding Layer: The first layer is an embedding layer. It converts word indices into dense vectors of fixed size (100 in this case). Set the input length to the maximum sequence length minus one, which corresponds to the number of previous words the model will consider when predicting the next word.

Bidirectional LSTM Layer: Add a Bidirectional LSTM layer with 150 units. This layer allows the model to learn context from both directions (past and future) in the sequence. return_sequences=True

Dropout Layer: Add a dropout layer with a rate of 0.2 to prevent overfitting by randomly setting 20% of the input units to 0 during training.

LSTM Layer: Add a second LSTM layer with 100 units. This layer processes the sequence and passes its output to the next layer.

Dense Layer (Intermediate): Add a dense layer with half the total number of words as units, using ReLU activation. A regularization term (L2) is added to prevent overfitting.

Dense Layer (Output): The final dense layer has as many units as there are words in the vocabulary, with a softmax activation function to output a probability distribution over all words.

In [ ]:
from tensorflow.keras.regularizers import l2

model = Sequential([

    # Your code here :
    Embedding(input_dim=total_words, output_dim=100, input_length=max_sequence_len - 1),
    Bidirectional(LSTM(150, return_sequences=True)),
    Dropout(0.2),
    LSTM(100),
    Dense(total_words // 2, activation='relu', kernel_regularizer=l2),
    Dense(total_words, activation='softmax')
])

model.build()

# Compile the Model:

Compile the model using categorical crossentropy as the loss function, the Adam optimizer for efficient training, and accuracy as the metric to evaluate during training.

In [ ]:
# Your code here :
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Print Model Summary:

Use model.summary() to print a summary of the model, which shows the layers, their output shapes, and the number of parameters.

In [ ]:
# Your code here :
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_7 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ bidirectional_7 (Bidirectional)      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_15 (LSTM)                       │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# sample_input = np.random.randint(0, total_words, (1, max_sequence_len - 1))

# # Use the model to build it
# model.predict(sample_input)

# # Now, call model.summary() again
# model.summary()

# Now train the model for 50 epochs and assign it to a variable called history.

Training the model with 50 epochs should get you around 40% accuracy.

You can train the model for as many epochs as you like depending on the time and computing constraints you are facing. Ideally train it for a larger amount of epochs than 50.

That way you will get better text generation at the end.

However, dont waste your time.

In [ ]:
# Your code here :
# Train the model for 50 epochs
history = model.fit(predictors, labels, epochs=5, batch_size=64, validation_split=0.2)

# Display the training history for accuracy
print("Training Accuracy:", history.history['accuracy'][-1])
print("Validation Accuracy:", history.history['val_accuracy'][-1])

Epoch 1/5
223/223 ━━━━━━━━━━━━━━━━━━━━ 235s 1s/step - accuracy: 0.0392 - loss: 5.9450 - val_accuracy: 0.0258 - val_loss: 7.2836
Epoch 2/5
223/223 ━━━━━━━━━━━━━━━━━━━━ 297s 1s/step - accuracy: 0.0528 - loss: 5.8533 - val_accuracy: 0.0278 - val_loss: 7.3168
Epoch 3/5
223/223 ━━━━━━━━━━━━━━━━━━━━ 280s 1s/step - accuracy: 0.0530 - loss: 5.8164 - val_accuracy: 0.0320 - val_loss: 7.4701
Epoch 4/5
223/223 ━━━━━━━━━━━━━━━━━━━━ 277s 1s/step - accuracy: 0.0527 - loss: 5.7619 - val_accuracy: 0.0424 - val_loss: 7.5453
Epoch 5/5
223/223 ━━━━━━━━━━━━━━━━━━━━ 247s 988ms/step - accuracy: 0.0556 - loss: 5.6934 - val_accuracy: 0.0365 - val_loss: 7.5013
Training Accuracy: 0.05475989729166031
Validation Accuracy: 0.03650660067796707


# Use plt from matplotlib to plot the training accuracy over epochs and the loss over epochs

First you will have to get the accuracy and loss data over epochs, you can do this by using methods on your model.

In [ ]:
# Your code here :
import matplotlib.pyplot as plt

# Plotting training and validation accuracy
plt.figure(figsize=(14, 5))

# Plot accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()


# Generate text with the model based on a seed text

Now you will create two variables :

- seed_text = 'Write the text you want the model to use as a starting point to generate the next words'
- next_words = number_of_words_you_want_the_model_to_generate

Please change number_of_words_you_want_the_model_to_generate by an actual integer.

In [ ]:
# Your code here :
seed_text = 'Write the text you want the model to use as a starting point to generate the next words'
next_words = 50

Now create a loop that runs based on the next_words variable and generates new text based on your seed_text input string. Print the full text with the generated text at the end.

This time you dont get detailed instructions.

Have fun!

In [ ]:
# Your code here :
def generate_text(seed_text, next_words):
  for _ in range(next_words):
      token_list = tokenizer.texts_to_sequences([seed_text])[0]
      token_list = pad_sequences([token_list], maxlen=max_sequence_len - 1, padding='pre')
      predicted = model.predict(token_list, verbose=0)
      predicted_word_index = np.argmax(predicted, axis=-1)[0]
      output_word = tokenizer.index_word[predicted_word_index]
      seed_text += " " + output_word

  return seed_text

In [ ]:
generate_text1 = generate_text(seed_text, next_words)
print(generate_text1)

Write the text you want the model to use as a starting point to generate the next words and i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i


Experiment with at least 3 different seed_text strings and see what happens!

In [ ]:
# Your code here :
seed_text2 = 'hey this is test for check to check the style of Shakespeare'
next_words2 = 50
generate_text2 = generate_text(seed_text2, next_words2)
print(generate_text2)

seed_text3 = 'Hello world!'
next_words3 = 50
generate_text3 = generate_text(seed_text3, next_words3)
print(generate_text3)

seed_text4 = 'The earth is round then how does one exist in every corner of the earth?'
next_words4 = 50
generate_text4 = generate_text(seed_text4, next_words4)
print(generate_text4)

hey this is test for check to check the style of Shakespeare the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love of the love
Hello world! the love and i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i
The earth is round then how does one exist in every corner of the earth? and art and i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i i
